# Ejercicio 4
#### Métodos computacionales:
##### Marsili - Caracoix - Wolodarsky

In [12]:
import pandas as pd
from matricesRalas import *

papers = pd.read_csv('papers/papers.csv', header = 0)
citas = pd.read_csv('papers/citas.csv', header = 0)

- Importamos las funciones de marticesRalas y la biblioteca de Pandas.
- Usamos pandas para extraer la informacion de los .csv

In [20]:
W = MatrizRala(len(papers), len(papers))

D = MatrizRala(len(papers), len(papers))

cantidad_citas_dadas = [0] * len(papers)

cantidad_citas_recibidas = [0] * len(papers)

for index, row in citas.iterrows():
    W[row['to'], row['from']] = 1
    cantidad_citas_dadas[row['from']] += 1
    cantidad_citas_recibidas[row["to"]] += 1

for i in range(len(cantidad_citas_dadas)):
    if cantidad_citas_dadas[i] != 0:
        D[i, i] = 1/cantidad_citas_dadas[i]

- Creamos las matrices W y D con tamaño len(papers) x len(papers)
- Construimos la matriz W con los valores de las citaciones según las asignaciones definidas en ‘assignments’
- Y luego la matriz D, donde cada entrada en la diagonal tiene la probabilidad de que ese paper sea citado: 1/𝑐𝑖, donde ci es el conjunto de índices de los papers que citan a 𝑝𝑖. Es decir, 𝑗 está en 𝐶(𝑖) si 𝑝𝑗 cita a 𝑝i.


In [21]:
def matriz_de_unos(n,m):
    matriz = MatrizRala(n,m)
    for i in range(matriz.shape[0]):
        for j in range(matriz.shape[1]):
            matriz[i,j] = 1
    return matriz

In [22]:
#Genero mi vector de probabilidades de manera iterativa
N = len(papers)
d = 0.85
p_t = MatrizRala(N,1)
for i in range(N):
    p_t[i,0] = 1/N

tolerance = 1e-6
error = 1

mat_unos = matriz_de_unos(N,1)
unoMenosDeSobreEne = ((1-d)/N) * mat_unos
d_W = d * W
d_WD = d_W @ D


while error > tolerance:
    # Multiplica la matriz W_D por el vector p_t y escala por d
    p_t_plus_1 = d_WD @ p_t
    p_t_plus_1 = unoMenosDeSobreEne + p_t_plus_1
    # Calcula el error máximo en esta iteración comparando el nuevo vector de PageRank con el anterior
    error = max(abs(p_t_plus_1[i,0] - p_t[i,0]) for i in range(N))

    # Actualiza el vector de PageRank para la próxima iteración
    p_t = p_t_plus_1

Estimamos el P* de forma iterativa, como en el ejercicio 3.

In [57]:
print("Top 10 Papers por Cantidad de Citas:")
# Crear una lista de (número de citas, índice)
lista_citas = [(cantidad_citas_recibidas[i], i) for i in range(len(papers))]
sorted_papers_citas = sorted(lista_citas, key=lambda x: x[0], reverse=True)
# Imprimir los top 10 papers por cantidad de citas recibidas con su información
for rank, (num_citas, paper_id) in enumerate(sorted_papers_citas[:10], start=1):
    paper = papers.loc[paper_id]
    print(f"{rank}. Paper ID: {paper['id']}, Title: \"{paper['titulo']}\", Número de Citas: {num_citas}")

print('\n')

print("Top 10 Papers por PageRank:")
# Crear lista de tuplas (puntaje de PageRank, índice)
lista = [(p_t[i, 0], i) for i in range(len(papers))]
sorted_papers = sorted(lista, key=lambda x: x[0], reverse=True)

# Crear un diccionario para búsqueda rápida de rankings
ranking_dict = {paper_id: rank for rank, (score, paper_id) in enumerate(sorted_papers, start=1)}

print(ranking_dict[453387])

for rank, (score, paper_id) in enumerate(sorted_papers[:10], start=1):
    paper = papers.loc[paper_id]
    # Obtenemos los papers que citan a este paper
    citadores = citas[citas['to'] == paper_id]['from']
    citadores_ranking = [(ranking_dict[citador], citador) for citador in citadores if citador in ranking_dict]

    # Ordenamos los citadores por su ranking
    citadores_ranking.sort()

    # Obtenemos los top 10 citadores
    top_citadores = citadores_ranking[:10]
    top_citadores_info = [f"Rank: {rank}, Paper ID {papers.at[citador, 'id']}, Title: \"{papers.at[citador, 'titulo']}\"" for rank, citador in top_citadores]

    print(f"{rank}. Paper ID: {paper['id']}, Title: \"{paper['titulo']}\", Score: {score:.6f}, Citas Recibidas: {cantidad_citas_recibidas[paper_id]}")
    print("   Top citadores:")
    for citer_info in top_citadores_info:
        print(f"     {citer_info}")

Top 10 Papers por Cantidad de Citas:
1. Paper ID: 453387, Title: "Introduction to algorithms", Número de Citas: 816
2. Paper ID: 162585, Title: "Compilers: principles, techniques, and tools", Número de Citas: 791
3. Paper ID: 214951, Title: "C4.5: programs for machine learning", Número de Citas: 648
4. Paper ID: 151297, Title: "Smalltalk-80: the language and its implementation", Número de Citas: 588
5. Paper ID: 326368, Title: "A relational model of data for large shared data banks", Número de Citas: 566
6. Paper ID: 319217, Title: "Time, clocks, and the ordering of events in a distributed system", Número de Citas: 560
7. Paper ID: 586607, Title: "The nature of statistical learning theory", Número de Citas: 553
8. Paper ID: 311413, Title: "Pattern Classification (2nd Edition)", Número de Citas: 524
9. Paper ID: 79620, Title: "The art of computer programming, volume 1 (3rd ed.): fundamental algorithms", Número de Citas: 522
10. Paper ID: 81323, Title: "The art of computer programming, v